<img src="./images/cmiyc2024_test.webp" alt="A kitten with a tophat at a computer. Aka JtR + H" width="800"/>

# Cracking CMIYC 2024 Test Hashes

**GETTING READY FOR CMIYC 2024!!!!**

### Overview

The goal of this notebook is to make it easy as possible for you to get a team registered and ready to participate in CMIYC. I'm hopefull I'll be updating it a lot running up to the contest, so I'd recommend using this as an example, but creating your own notebook to manage your teams cracking sessions.

### Major Features and Training Goals

1. How to register a team with CMIYC
2. TBD. Still need to work on this --Lakiw



### Creating a PGP Key (If Needed)

Let's face it. PGP is a pain, and you don't want to have to try and figure it out during the contest. But all communicaiton with the organizers (KoreLogic) need to be done via PGP encrypted messages. This includes creating a team and submitting hashes. To make this easier, I created some functions to help using the Python3 **pgpy** library.

The first function is **generate_read_pgp_private_key**. This will create a new PGP key (if one does not already exit) and write it to the key_file name that you send to it. If a PGP key already exists it'll validate the key and then return it. This way you aren't creating a new key every time this is run.

In [1]:
from lib_framework.pgp_mgr import generate_read_pgp_private_key

import os
import sys

key_file = os.path.join(
    '.',
    'challenge_files',
    'CMIYC2024_Test',
    'pgp_key.asc'
    )

my_private_key = generate_read_pgp_private_key(key_file, email_address="cweir@vt.edu", team_name="Reusablesec")

### Creating a Registration Email

To make it easier to encrypt/decrypt messages I created the **PGP_Mgr** class. I figured a class would be better than having individual functions since I didn't want to have to keep passing in my private key as well as KoreLogic's public key.

While I could try to integrate this with individual e-mail services, that seems like a lot of work so PGP_Mgr will output encrypted messages to this workbook. It's up to you then to actually copy/paste that message into an e-mail and send it to KoreLogic at sub-2024@contest.korelogic.com

Note, the register_team function will both print the message to your screen as well as return the message. Given the way JupyterLab displays newlines in returned messages I'd recommend hiding the return value using the **";"** at the end of the function call, and copy/pasting the printed version.

In [2]:
from lib_framework.pgp_mgr import PGPMgr, read_pgp_key

# Load up KoreLogic's submisison public key
korelogic_sub_public_key_file = os.path.join(
    '.',
    'challenge_files',
    'CMIYC2024_Test',
    'defcon-contest-sub-2024.pub.asc'
    )
korelogic_sub_public_key = read_pgp_key(korelogic_sub_public_key_file)

default_attachment = os.path.join(
    '.',
    'challenge_files',
    'CMIYC2024_Test',
    'submission.asc'
    )

# Create the PGPMgr class
pgp_mgr = PGPMgr(my_private_key, korelogic_sub_public_key, default_attachment)

# Create Submission e-mail
#pgp_mgr.register_team("Someone Didn't Update Their Teamname in the JupyterLab Framework by Lakiw");
pgp_mgr.register_team("Reusablesec");

-----BEGIN PGP MESSAGE-----

wcBMA3EnRIyYyGY/AQgAkZJRenAILqgdaVYWpOpUFZA2zhocrYF+KwyGo0mL98I8
c3Df7/hoBfwTL/nY7igGKvH3GFv2z52IDDQHDptDJy6IhjgOiPjVO7vLsOJ0pdfS
N65gRfzsjSuWbwkq5nh283TY8IUzi51+vZ2LskQ8zS3z1yxLXWFA0BkHmPRx27zo
//SeiESR2UupG+Z4TsDjNhTEbbkTf1WOfIzWqCKgI1o/BDtPCMlxmVsulQ0eyyP4
Jo/6D6JfMdqumUZ685Vb21DOK4EDJbArBwOCGlQc0HCe+XdyHiSrgdLA/JgIAr8K
pWrD7TihCKC1qRXYeNWCBPdlOjXEW++8uPdPbtwYSNLHTQGrVgYaVCP7fwv8Ankv
QuqcvPZrUVu5sySZ1C6mgOD7D7j34A2z1DAGI7Hyu9wNY9+xAWkt1gWPgWBXmjVd
Pw0JF7pr1mYUB8PNxz4Of5hkLYkS43KBeNgoYbN6nl3y+yDCP9cCrTQJ79rqbE/r
jgoMeihIlLtjLXEmGRROw2OePFCbC2p3LHW/VYdARfKUdbFERO34SrRDsQPO6ASm
ohIowybyR2qYtF7o12FX/R1tL0oRuRJ9cl8B/xrFmFo/P5h957r9Sl/em+EUDxh/
G2Y3bG6GlBhVwxKzXcER1jTntMNDt1Ela3S33UPo4nVziplpS5ketlh/772ziDKw
pi3wyNhRk4hXJls9Q2S48d9kxzABrw0MLc1iPGsW003KgE+V4na5Z5gLKEycO3Nt
owLheaapnu1yqa7OsTDQSS32b2QfpQKPwvA322N1htr9FGxvKxSD4YEArF4r1oT2
GFAPDZdWoUWj1B4SN832vbYiVERKlZrKtcBC2WdqM7U2SpBzAj4slIcGE02oIdK9
3gB/rMwuadl/gB5TA38p2YKjGc+WLqVrilrT8h5/seE0IVugolcHL3AEFuBEH

In [6]:
# Now we need to validate the confirmation e-mail
default_attachment = os.path.join(
    '.',
    'challenge_files',
    'CMIYC2024_Test',
    'korelogic_comms.asc'
    )
challenge_response = pgp_mgr.decrypt_msg(default_attachment)

Thank you, your key has been successfully confirmed.

You can start submitting cracks as soon as the contest has started.

Keep an eye on the contest website; we may release some test hashes
a few days before the contest, so teams can confirm their submission
process works.

Don't forget to let us know if you want to compete as a 'Pro' team.




In [5]:
pgp_mgr.encrypt_msg(challenge_response)

-----BEGIN PGP MESSAGE-----

wcBMA3EnRIyYyGY/AQf9EkVIvM1jDw7Gs80lxEa5noovNNadNfMqvNiaTrjDVDmX
klfZeKQTyVeSaRudL13W+454HwWBqeu7MUv/70DVRSk7BDNzOB8hF7CzUlHY2roi
zqnNLvrJl2x7EaSQAUUyci81It4vIRTqn7FvspigiSzn1tOZ2YV2wdOHKijRMI4z
piMi1KhQk0IakTLTyJVHgK0veRalfRhOy+M2ac+RyT1Ve9/5Yxx5Bzr/yK1X4d3e
pg6aO2ZBBt99XZvLwXyWiJZJmzfVkfxTvYMSz/xNivNg83h4yYxKFj2fX9aeAtxy
hKo0ThWWaCHyaMMTwPg7TIK+fZbCkIeMh44CahiYyNLB2QETLXAgS5oQWQePdimA
QCoR/EF42opmbe29qGSnnU0mV4Oamk9BNtYMYmAsip+k+gzgWkmITPAkyoJM8RN9
ONp0trimATTu9u3/vlIKSsSTQRjBTkgObO15403W1YKBcDyk/w1fJV90AVqO7x04
GEs32WaLDw/QcI1dKnQMq4hsTAdeOAke9F7rU2ZCg+hj9+IjbSQANonUedSk83Kd
EqxHHYnLS3pFOO9qQzFtaaYGoNA5ldfvaacI8S/P7GaPsRS+7CzMLaRMZY2rnwF+
sQumHRPDi2pWVKnCxIBNKasliU4Fzr1PEHlYqqGX1dAbl5QgRH7RxDPp2fK8j1JW
kgAEjTriKGp73R73ufVqj8l2yUM+1H/uv4ZE7BpEAvCaPkfA/X93K1QB+MeLHzxM
jIS4DUuDE1c64fuV2xBSjb6OeKPxWNrbREfXBlo1rAyGCwwWvlxDEnlc51e9tykW
ntFdkxWxg6M/WwG+ZqOP/JYYx5psiFO842OQUtjURcP+I5moMUtX5fEnhoZ0GN5P
TVFt7fF+MtLokB/WC8Xb0yso9mvIKKNvVEwU7SEdDMirhhxfDohDwoIgSiBoa